In [2]:
!pip uninstall speac

Uninstalling speac-1.0.13:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/speac-1.0.13.dist-info/*
    /usr/local/lib/python3.7/dist-packages/speac/*
Proceed (y/n)? y
  Successfully uninstalled speac-1.0.13


In [3]:
!pip install speac

In [5]:
import speac
import music21

from speac.chopin_33_3 import CHOPIN_33_3
from speac.chopin_63_2 import CHOPIN_63_2
from speac.chopin_67_4 import CHOPIN_67_4

from speac.top_level import *
from music21 import *

In [6]:
import os
import pprint
import glob
import numpy
from tensorflow import keras
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.utils import np_utils
import timeit

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Чтение хоралов Баха и их запись в форму, удобную для работы со SPEAC-анализом

In [8]:
  chorales = []
  signatures = []
  k = 1

  majors = dict([("A-", 4),("G#", 4),("A", 3),("A#", 2),("B-", 2),("B", 1),
                 ("C", 0),("C#", -1),("D-", -1),("D", -2),("D#", -3),("E-", -3),
                 ("E", -4),("F", -5),("F#", 6),("G-", 6),("G", 5)])   

  minors = dict([("G#", 1), ("A-", 1),("A", 0),("A#", -1),("B-", -1),("B", -2),
                 ("C", -3),("C#", -4),("D-", -4),("D", -5),("D#", 6),("E-", 6),
                 ("E", 5),("F", 4),("F#", 3),("G-", 3),("G", 2)])

  for file in glob.glob("/content/drive/My Drive/Examination/MIDI/chorales/*.mid*"):
    print("Reading file №" + str(k) + "; " + file)
    
    events = []

    k= k + 1
    main_offset = None

    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)

    key = midi.analyze('key')
    print("key before = " + key.tonic.name, key.mode)

    if key.mode == "major":
        halfSteps = majors[key.tonic.name]

    elif key.mode == "minor":
        halfSteps = minors[key.tonic.name]

    midi = midi.transpose(halfSteps)
    key = midi.analyze('key')
    print("key after  =  " + key.tonic.name, key.mode)  

    if parts: # Если есть хотя бы 1 партия
        notes_to_parse = parts.parts[0].recurse()

    else: # file has notes in a flat structure  (не понял для чего эта ветка)
        notes_to_parse = midi.flat.notes

    #<music21.meter.TimeSignature 4/4> -> 4/4
    if midi.flat.timeSignature is not None :
      signature = str(midi.flat.timeSignature).split(" ")[1].split(">")[0].split("/")[0]
      signatures.append(signature)
    else:
      signatures.append(None)
    
    i = 1
    for element in notes_to_parse:
        if main_offset is None and (isinstance(element, note.Note) or isinstance(element, chord.Chord)):
            if element.offset == 0:
                main_offset = 0
            else:
                main_offset = element.offset

        if isinstance(element, note.Note): #Записываем ноту
            event = []
            event.append(int((element.offset - main_offset) * 1000))
            event.append(element.pitch.midi)
            event.append(int(element.duration.quarterLength * 1000))
            event.append(1)
            event.append(element.volume.velocity)
            events.append(event)

        elif isinstance(element, chord.Chord):#Записываем аккорд
            for e in element:
                if isinstance(e, note.Note):
                    event = []
                    event.append(int((element.offset - main_offset) * 1000))
                    event.append(e.pitch.midi)
                    event.append(int(e.duration.quarterLength * 1000))
                    event.append(1)
                    event.append(e.volume.velocity)
                    events.append(event)
    
    chorales.append(events)

Reading file №1; /content/drive/My Drive/Examination/MIDI/chorales/04esistd.mid
key before = E major
key after  =  C major
Reading file №2; /content/drive/My Drive/Examination/MIDI/chorales/10austie.mid
key before = A minor
key after  =  A minor
Reading file №3; /content/drive/My Drive/Examination/MIDI/chorales/01ausmei.mid
key before = G major
key after  =  C major
Reading file №4; /content/drive/My Drive/Examination/MIDI/chorales/03achgot.mid
key before = A minor
key after  =  A minor
Reading file №5; /content/drive/My Drive/Examination/MIDI/chorales/02ichdan.mid
key before = A major
key after  =  C major
Reading file №6; /content/drive/My Drive/Examination/MIDI/chorales/09ermunt.mid
key before = G major
key after  =  C major
Reading file №7; /content/drive/My Drive/Examination/MIDI/chorales/12puerna.mid
key before = A minor
key after  =  A minor
Reading file №8; /content/drive/My Drive/Examination/MIDI/chorales/06christ.mid
key before = F major
key after  =  C major
Reading file №9;

In [17]:
print(chorales[0][:9])

[[0, 67, 1000, 1, 64], [0, 64, 1000, 1, 64], [0, 60, 1000, 1, 64], [0, 48, 1000, 1, 64], [1000, 67, 1000, 1, 64], [1000, 62, 500, 1, 64], [1000, 62, 1000, 1, 64], [1000, 47, 1000, 1, 64], [1500, 64, 500, 1, 64]]


Для каждого хорала выполняется SPEAC-анализ, после чего в каждое событие каждого хорала добавляется 3 SPEAC-символа: foreground, middleground, background.

In [10]:
meters = []
speac_analyzes = []

for i in range(1, len(chorales) + 1):
  events = chorales[i - 1]

  meter = 3
  while meter < 21:
      try:
        result = get_note_and_speac(events, meter)
        speac_analyzes.append(result)
        print("SPEAC-analysis for file №", i)
        meters.append(meter)
        break

      except Exception as e:
        meter += 1
      pass
  print("------------------")

SPEAC-analysis for file № 1
------------------
SPEAC-analysis for file № 2
------------------
SPEAC-analysis for file № 3
------------------
SPEAC-analysis for file № 4
------------------
SPEAC-analysis for file № 5
------------------
SPEAC-analysis for file № 6
------------------
SPEAC-analysis for file № 7
------------------
SPEAC-analysis for file № 8
------------------
SPEAC-analysis for file № 9
------------------
SPEAC-analysis for file № 10
------------------
SPEAC-analysis for file № 11
------------------
SPEAC-analysis for file № 12
------------------
SPEAC-analysis for file № 13
------------------
SPEAC-analysis for file № 14
------------------
SPEAC-analysis for file № 15
------------------
SPEAC-analysis for file № 16
------------------
SPEAC-analysis for file № 17
------------------
SPEAC-analysis for file № 18
------------------
SPEAC-analysis for file № 19
------------------
SPEAC-analysis for file № 20
------------------
SPEAC-analysis for file № 21
------------------
S

События последовательно записываются друг за другом

In [15]:
one_big_input = []
big_offset = 0

for speac_analysis in speac_analyzes:
    for event in speac_analysis:
            new_event = copy.deepcopy(event)
            new_event[0] += big_offset
            one_big_input.append(new_event)
    
    big_offset += speac_analysis[-1][0] + speac_analysis[-1][2]

print(one_big_input[:9])

[[0, 48, 1000, 1, 64, 'statement', 'preparation', 'statement'], [0, 60, 1000, 1, 64, 'statement', 'preparation', 'statement'], [0, 64, 1000, 1, 64, 'statement', 'preparation', 'statement'], [0, 67, 1000, 1, 64, 'statement', 'preparation', 'statement'], [1000, 47, 500, 1, 64, 'extension', 'preparation', 'statement'], [1000, 62, 500, 1, 64, 'extension', 'preparation', 'statement'], [1000, 62, 500, 1, 64, 'extension', 'preparation', 'statement'], [1000, 67, 500, 1, 64, 'extension', 'preparation', 'statement'], [1500, 47, 500, 1, 64, 'extension', 'preparation', 'statement']]


Кодирование каждого бита (одновременно играющихся событий) в формате: "midi.midi.midi", предложенном в статье https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

In [14]:
notes_article = []
current_chord = None

last_event_offset = None

for event in one_big_input:
    event_offset = event[0]
    
    if last_event_offset is None:
        current_chord = str(event[1])
    
    elif event_offset > last_event_offset:
        notes_article.append(current_chord)
        current_chord = str(event[1])

    else:
        current_chord = current_chord + "." + str(event[1])

    last_event_offset = event_offset
    
print(notes_article[:9])

['48.60.64.67', '47.62.62.67', '47.62.64.67', '43.60.65.67', '43.59.65.67', '48.60.64.67', '50.53.62.70', '52.55.62.70', '53.57.60.69']


Если после установки что-то в коде не работает, то есть вероятность, что надо сбросить настройки в среде выполнения или изменить её на TPU соответственно)))


In [ ]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.88.227.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.88.227.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


TPU имеет 8 логических устройств TPU (0–7), которые могут выполнять параллельную обработку. Следовательно, мы определяем стратегию распределения для распределенного обучения по этим 8 устройствам:


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Подготовка данных для обучения модели из статьи

In [ ]:
#Минимальное необходимое число нот в обучающих произведениях
sequence_length = 8

#Сортировка всех повторяющихся нот в множество неповторяющихся (Set)
pitchnames_article = sorted(set(item for item in notes_article))

#Отображаем каждый аккорд или ноту на некоторое число (по сути Map)
note_to_int_article = dict((note, number) for number, note in enumerate(pitchnames_article))

network_input_article = []
network_output_article = []

for i in range(0, len(notes_article) - sequence_length, 1):
    #записываем сначала первые 100 нот, затем 1-101, 2-102 и т.д
    sequence_in_article = notes_article[i:i + sequence_length]

    #узнаём какой должна быть следующая нота
    sequence_out_article = notes_article[i + sequence_length]

    #ноты преобразуем в номера из созданного Map
    network_input_article.append([note_to_int_article[char] for char in sequence_in_article])

    #преобразуем следующую ноту в номер из созданного Map
    network_output_article.append(note_to_int_article[sequence_out_article])

n_patterns_article = len(network_input_article)

#Преобразование входной выборки в формат, удобный для работы с LSTM
network_input_article = numpy.reshape(network_input_article, (n_patterns_article, sequence_length, 1))

n_vocab_article = len(set(notes_article))
# n_vocab = max(set(network_output)) + 1

#Нормализация
network_input_article = network_input_article / float(n_vocab_article)

network_output_article = np_utils.to_categorical(network_output_article)

print(network_input_article[0])

[[0.26683292]
 [0.21030756]
 [0.21113882]
 [0.08312552]
 [0.07813799]
 [0.26683292]
 [0.32751455]
 [0.46716542]]


In [ ]:
def create_model(network_input, n_vocab):
  model = Sequential()
  model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], 
                     network_input.shape[2]),
        return_sequences=True
    ))
  model.add(Dropout(0.3))
  model.add(LSTM(512, return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(256))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))

  model.summary()

  return model

Компиляция модели

In [ ]:
with strategy.scope():
    model_article = create_model(network_input_article, n_vocab_article)
    model_article.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    print("model_article is compilled")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 8, 256)            264192    
_________________________________________________________________
dropout (Dropout)            (None, 8, 256)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 8, 512)            1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 512)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

Обучение модели

In [ ]:
model_article.fit(network_input_article, network_output_article, epochs=200, batch_size=64)    

Epoch 1/200
43/43 [==============================] - 29s 79ms/step - loss: 7.0183
Epoch 2/200
43/43 [==============================] - 1s 16ms/step - loss: 6.7539
Epoch 3/200
43/43 [==============================] - 1s 16ms/step - loss: 6.7263
Epoch 4/200
43/43 [==============================] - 1s 16ms/step - loss: 6.6173
Epoch 5/200
43/43 [==============================] - 1s 16ms/step - loss: 6.6102
Epoch 6/200
43/43 [==============================] - 1s 17ms/step - loss: 6.5597
Epoch 7/200
43/43 [==============================] - 1s 16ms/step - loss: 6.4912
Epoch 8/200
43/43 [==============================] - 1s 16ms/step - loss: 6.5048
Epoch 9/200
43/43 [==============================] - 1s 16ms/step - loss: 6.4285
Epoch 10/200
43/43 [==============================] - 1s 16ms/step - loss: 6.4167
Epoch 11/200
43/43 [==============================] - 1s 16ms/step - loss: 6.4099
Epoch 12/200
43/43 [==============================] - 1s 16ms/step - loss: 6.3338
Epoch 13/200
43/43 [====

In [ ]:
model_article.save_weights('/content/drive/My Drive/Examination/Weights/model_article_weights.h5', overwrite=True)

Генерация следующих битов

In [ ]:
  # start = numpy.random.randint(0, len(network_input_article)-1)
  start = 0
  int_to_note_article = dict((number, note) for number, note in enumerate(pitchnames_article))

  pattern_article = network_input_article[start]

  prediction_output_article = []

  # generate n notes
  for note_index in range(50):
        print("generating ", note_index, " note")
        prediction_input_article = numpy.reshape(pattern_article, (1, len(pattern_article), 1))
        prediction_input_article = prediction_input_article / float(n_vocab_article)

        prediction_article = model_article.predict(prediction_input_article, verbose=0)

        index_article = numpy.argmax(prediction_article)
        result_article = int_to_note_article[index_article]
        prediction_output_article.append(result_article)

        pattern_article = numpy.append(pattern_article, [index_article])
        pattern_article = pattern_article[1:len(pattern_article)]

generating  0  note
generating  1  note
generating  2  note
generating  3  note
generating  4  note
generating  5  note
generating  6  note
generating  7  note
generating  8  note
generating  9  note
generating  10  note
generating  11  note
generating  12  note
generating  13  note
generating  14  note
generating  15  note
generating  16  note
generating  17  note
generating  18  note
generating  19  note
generating  20  note
generating  21  note
generating  22  note
generating  23  note
generating  24  note
generating  25  note
generating  26  note
generating  27  note
generating  28  note
generating  29  note
generating  30  note
generating  31  note
generating  32  note
generating  33  note
generating  34  note
generating  35  note
generating  36  note
generating  37  note
generating  38  note
generating  39  note
generating  40  note
generating  41  note
generating  42  note
generating  43  note
generating  44  note
generating  45  note
generating  46  note
generating  47  note
ge

In [ ]:
import music21
from music21 import *

In [ ]:
def create_new_midi(predicted, file_name):
    offset = 0
    output_notes = []
    # create note and chord objects based on the values generated by the model

    for pattern in predicted:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            note_index = 0

            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)

            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)

        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
                    
            output_notes.append(new_note)

        offset += 1.0

    midi_stream = stream.Stream(output_notes)
    file = midi_stream.write('midi', fp=file_name)

In [ ]:
create_new_midi(prediction_output_article, "article_n=" + str(sequence_length) + ".mid")

Попытаемся расширить входной вектор так, чтобы на вход нейросети подавался не скаляр, а вектор из 2-х переменных: аккорды и спеак. Будем пытаться повторять эксперимент, который проводил Коуп, когда генерировал музыку. То есть он на основе одного аккорда и желаемого следующего SPEAC-состояния по определённому алгоритму генерировал следующий аккорд. Мы делаем то же самое, но вместо алгоритма Коупа будем использовать нейронную сеть.
Здесь стоит учесть, что при кодировке всех трёх символов одним разница между foreground состояниями должна быть минимальной, то есть нужно правильно отсортировать данные.



In [ ]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.88.227.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.88.227.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Сортировка SPEAC-состояний по мере напряженности аккорда 

In [18]:
speac_symbols = ["antecedent", "preparation", "extension", "statement", "consequent"]

In [19]:
speac_map_number = 1
sorted_speac_to_int = {}

for background in speac_symbols:

    for middleground in speac_symbols:

        for foreground in speac_symbols:
            speac_string = str(foreground) + "-" + str(middleground) + "-" + str(background)
            sorted_speac_to_int[speac_string] = speac_map_number
            speac_map_number += 1   

Подготовка данных для обучения 

In [21]:
notes = []
note = []
one_time_notes_set = set()

last_offset = None
one_time_notes = None

for event in one_big_input:
    if last_offset is None:
        one_time_notes = str(event[1])
    
    elif event[0] == last_offset:
        one_time_notes = one_time_notes + "." + str(event[1])
    
    else:
        next_speac = str(event[5]) + "-" + str(event[6]) + "-" + str(event[7])
        note = [one_time_notes, next_speac]
        one_time_notes_set.add(one_time_notes)
        notes.append(note)
        one_time_notes = str(event[1])

    last_offset = event[0]

print(notes[:9])

[['48.60.64.67', 'extension-preparation-statement'], ['47.62.62.67', 'extension-preparation-statement'], ['47.62.64.67', 'antecedent-preparation-statement'], ['43.60.65.67', 'antecedent-preparation-statement'], ['43.59.65.67', 'statement-preparation-statement'], ['48.60.64.67', 'preparation-preparation-statement'], ['50.53.62.70', 'preparation-preparation-statement'], ['52.55.62.70', 'extension-preparation-statement'], ['53.57.60.69', 'extension-preparation-statement']]


In [ ]:
#Минимальное необходимое число нот в обучающих произведениях
sequence_length = 38
sequence_length += 1

#Сортировка всех повторяющихся нот в множество неповторяющихся (Set)
notes_pitchnames = sorted(set(item for item in one_time_notes_set))

#Отображаем каждый аккорд или ноту на некоторое число (по сути Map)
notes_to_int = dict((note, number) for number, note in enumerate(notes_pitchnames))

network_input = []
network_input_without_normalisation = []
network_output = []

n_vocab_notes = len(one_time_notes_set)
n_vocab_speac = len(sorted_speac_to_int)

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]

    # Берём только ноты, без speac
    sequence_out_notes = notes[i + sequence_length][0]  

    local_network_input = []
    local_network_input_without_normalisation = []
    for j in range(1, sequence_length):
        notes_in_input = sequence_in[j - 1][0]
        speac_in_input = sequence_in[j][1]      # Подаём не текущий speac, а следующий

        notes_to_number = notes_to_int[notes_in_input]
        speac_to_number = sorted_speac_to_int[speac_in_input]

        normalized_notes_to_number = notes_to_number / float(n_vocab_notes)  
        normalized_speac_to_number = speac_to_number / float(n_vocab_speac) 

        local_network_input.append([normalized_notes_to_number, normalized_speac_to_number])
        local_network_input_without_normalisation.append([notes_to_number, normalized_speac_to_number])

    network_input.append(local_network_input)
    network_input_without_normalisation.append(local_network_input_without_normalisation)

    network_output.append(notes_to_int[sequence_out_notes])

print("normalized network input[0]:")
print(network_input[0])
print("----------------")

n_patterns = len(network_input)
sequence_length -= 1

#Преобразование входной выборки в формат, удобный для работы с LSTM
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 2))  # подавать в нейросеть данные по 2 элемента
print("reshaped network input[0]:")
print(network_input[0])
print("----------------")

network_output = np_utils.to_categorical(network_output)

normalized network input[0]:
[[0.2666112956810631, 0.7407407407407407], [0.2101328903654485, 0.7314814814814815], [0.21096345514950166, 0.7314814814814815], [0.08305647840531562, 0.7453703703703703], [0.07807308970099668, 0.7361111111111112], [0.2666112956810631, 0.7361111111111112], [0.3272425249169435, 0.7407407407407407], [0.46677740863787376, 0.7407407407407407], [0.574750830564784, 0.7407407407407407], [0.5872093023255814, 0.7407407407407407], [0.5980066445182725, 0.7407407407407407], [0.2707641196013289, 0.7453703703703703], [0.2599667774086379, 0.7314814814814815], [0.04152823920265781, 0.75], [0.19186046511627908, 0.75], [0.24086378737541528, 0.7453703703703703], [0.33637873754152825, 0.7453703703703703], [0.4601328903654485, 0.7407407407407407], [0.45598006644518274, 0.7407407407407407], [0.10382059800664452, 0.7407407407407407], [0.32475083056478404, 0.7361111111111112], [0.23338870431893688, 0.7361111111111112], [0.17857142857142858, 0.7361111111111112], [0.11129568106312292

In [ ]:
def create_new_model(network_input, n_vocab):
  model = Sequential()
  model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], 
                     network_input.shape[2]),
        return_sequences=True
    ))
  model.add(Dropout(0.3))
  model.add(LSTM(512, return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(256))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))

  model.summary()

  return model

In [ ]:
with strategy.scope():
    model_only_notes = create_new_model(network_input, n_vocab_notes)
    model_only_notes.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    print("model_only_notes is compilled")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 38, 256)           265216    
_________________________________________________________________
dropout_3 (Dropout)          (None, 38, 256)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 38, 512)           1574912   
_________________________________________________________________
dropout_4 (Dropout)          (None, 38, 512)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)              

In [ ]:
model_only_notes.fit(network_input, network_output, epochs=200, batch_size=64)    

Epoch 1/200
43/43 [==============================] - 13s 92ms/step - loss: 7.0475
Epoch 2/200
43/43 [==============================] - 1s 27ms/step - loss: 6.7892
Epoch 3/200
43/43 [==============================] - 1s 27ms/step - loss: 6.7151
Epoch 4/200
43/43 [==============================] - 1s 27ms/step - loss: 6.6115
Epoch 5/200
43/43 [==============================] - 1s 27ms/step - loss: 6.5541
Epoch 6/200
43/43 [==============================] - 1s 27ms/step - loss: 6.5010
Epoch 7/200
43/43 [==============================] - 1s 27ms/step - loss: 6.4712
Epoch 8/200
43/43 [==============================] - 1s 27ms/step - loss: 6.4187
Epoch 9/200
43/43 [==============================] - 1s 27ms/step - loss: 6.2985
Epoch 10/200
43/43 [==============================] - 1s 26ms/step - loss: 6.2907
Epoch 11/200
43/43 [==============================] - 1s 27ms/step - loss: 6.2353
Epoch 12/200
43/43 [==============================] - 1s 27ms/step - loss: 6.0816
Epoch 13/200
43/43 [====

In [ ]:
  # start = numpy.random.randint(0, len(network_input)-1)
  start = 0 # выбираем индекс, начиная с которого для генерации будет использоваться существующая структура 
  pattern = copy.deepcopy(network_input[start]) # первые N нот не генерируются, а берутся из этого произведения

  int_to_note = dict((number, note) for number, note in enumerate(notes_pitchnames))

  predicted_outputs = []
  predicted_speac = []

  notes_count = 50
  # generate n notes
  for note_index in range(notes_count):
        # print("------------")
        print("generating ", note_index, " note")
        
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 2))
        prediction_input = copy.deepcopy(prediction_input)
        # print("prediction_input1 = ", prediction_input)
        # print("pattern1 = ", pattern)

        for i in range(1, len(prediction_input[0]) + 1):
            prediction_input[0][i - 1][0] /= float(n_vocab_notes) 
            prediction_input[0][i - 1][1] /= float(n_vocab_speac) 

        # print("prediction_input2 = ", prediction_input)
        # print("pattern2 = ", pattern)

        prediction_only_notes = model_only_notes.predict(prediction_input, verbose=0)

        index_only_notes = numpy.argmax(prediction_only_notes)
        result_only_notes = int_to_note[index_only_notes]
        predicted_outputs.append(result_only_notes)

        next_speac = notes[start + sequence_length + note_index][1] 
        speac_to_number = sorted_speac_to_int[next_speac]
        predicted_speac.append(next_speac)

        pattern = numpy.append(pattern, [[index_only_notes, speac_to_number]], axis=0) 
        # pattern = numpy.append(pattern, [[index_only_notes / float(n_vocab_notes) , speac_to_number /float(n_vocab_speac)]], axis=0) # тут точно не надо делить???
        pattern = pattern[1:len(pattern)]
        # print("pattern3 = ", pattern)

        

generating  0  note
generating  1  note
generating  2  note
generating  3  note
generating  4  note
generating  5  note
generating  6  note
generating  7  note
generating  8  note
generating  9  note
generating  10  note
generating  11  note
generating  12  note
generating  13  note
generating  14  note
generating  15  note
generating  16  note
generating  17  note
generating  18  note
generating  19  note
generating  20  note
generating  21  note
generating  22  note
generating  23  note
generating  24  note
generating  25  note
generating  26  note
generating  27  note
generating  28  note
generating  29  note
generating  30  note
generating  31  note
generating  32  note
generating  33  note
generating  34  note
generating  35  note
generating  36  note
generating  37  note
generating  38  note
generating  39  note
generating  40  note
generating  41  note
generating  42  note
generating  43  note
generating  44  note
generating  45  note
generating  46  note
generating  47  note
ge

In [ ]:
import music21
from music21 import *

In [ ]:
create_new_midi(predicted_outputs, "speac_example_n=" + str(sequence_length) + ".mid")
